In [ ]:
import os
import sys
import json
import numpy as np
import time
import cv2
from PIL import Image, ImageDraw
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapOnImage
import tensorflow as tf


In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = './'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist.'

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class SingleNageurConfig(Config):
    """Configuration pour un entrainement fait sur un nageur.
    Model ispiré de l'original mais adapté à mon GPU
    """
    NAME = "nageur"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Nombres de Classes
    NUM_CLASSES = 2 + 1  # background + 2 (head + arms)

    # Shap
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    #utili
    STEPS_PER_EPOCH = 500

    VALIDATION_STEPS = 5
    
    # Si le GPU le permet on peut utiliser un resnet 101, mais le 50 est efficace
    BACKBONE = 'resnet50'

    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = SingleNageurConfig()
config.display()

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            #if class_id < 1:
            #    print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
            #    return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            print (image_id)
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    print (image_file_name)
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

In [ ]:
dataset_val = CocoLikeDataset()
dataset_val.load_data(ROOT_DIR + '/datasets/nageur/val/coco_annotations.json', ROOT_DIR + '/datasets/nageur/val/images')
dataset_val.prepare()

In [ ]:
class InferenceConfig(SingleNageurConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    #seuil élevé pour réduire les faux positifs
    DETECTION_MIN_CONFIDENCE = 0.95
    

inference_config = InferenceConfig()

In [ ]:
# mode détection
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

In [ ]:
#get path pour le dernier entrainement
model_path = model.find_last()

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon  

def display_masks(to_display, titles):
    cols = 1
    rows = len(to_display)
    i=1
    cmap=None
    norm=None
    interpolation=None

    for image, title in zip(to_display, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        print()
        i += 1
    plt.show()   

In [ ]:
#Fonction générant des points depuis un masks
def getPoints(mask):
    height = mask.shape[0]
    width = mask.shape[1]
    points = []

    #######Premières positition de coude potentielle#####
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=i*4+j**4
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)
    #######Autres positition de coude potentielle (2)#####
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=(height-i)*4+j**4
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j
    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)
    
    #######Autres positition de coude potentielle (3)#####
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=i*4+(width-j)**4
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)
    #######Autres positition de coude potentielle (4)#####
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=(height-i)*4+(width-j)**4
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j
    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)    
    
    ######points les plus éloignées de y######
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=i
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j
    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)

    ######Points les plus éloignées sur x###############
    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):        
        for j in range (width):
            if (mask[i][j]==1):
                val=j
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j

    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)

    min=10000000000
    max=0                                                                                                                                                           
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):
        for j in range (width):
            if (mask[i][j]==1):
                val=i*i*i*j
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j

    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)



    min=10000000000
    max=0
    val=0
    iMin, iMax, jMin, jMax = 0, 0, 0, 0
    for i in range (height):
        for j in range (width):
            if (mask[i][j]==1):
                val=i*j*j*j
                if val < min:
                    min = val
                    iMin = i
                    jMin = j

                elif val > max:
                    max = val
                    iMax = i
                    jMax = j

    
    pointMin = []  
    pointMax = []    
    pointMin.append(iMin)
    pointMin.append(jMin)     
    pointMax.append(iMax)
    pointMax.append(jMax)    
    points.append(pointMin)
    points.append(pointMax)
    
    return points

In [ ]:
#fonction qui peut être utilisé pour supprimé des doublons mais n'est plus utile dans la dernière version.
def triTab (tab, h, w):
    N = len(tab)    
    tab_trie = []
    
    idDoublon = 0
    nbDoublon = 0
    for i in range(N-nbDoublon):
        doublon = False
        for j in range(i+1,N-nbDoublon):            
            if (tab[i][0] == tab[j][0]) and (tab[i][1] == tab[j][1]):
                doublon = True
                idDoublon = j
        if doublon :
            del tab[idDoublon]
            nbDoublon += 1
        point = []
        if i < len(tab) :
            point.append(tab[i][0])
            point.append(tab[i][1])
            tab_trie.append(point)
    N = len(tab) 
    tab = tab_trie
    
    tab_trie = []
    for i in range(N-nbDoublon):
        doublon = False
        for j in range(i+1,N-nbDoublon):            
            if (tab[i][0] == tab[j][0]) and (tab[i][1] == tab[j][1]):
                doublon = True
                idDoublon = j
        if doublon :
            del tab[idDoublon]
            nbDoublon += 1
        point = []
        if i < len(tab) :
            point.append(tab[i][0])
            point.append(tab[i][1])
            tab_trie.append(point)
       
    return tab_trie
                

In [ ]:
#permet de visualiser des points créé.
def modelisePoints(points, h, w) :
    
    model=np.zeros(shape=(h, w, 1), dtype=np.float32)        
    for i in range (len(points)):
        y = points[i][1]
        x = points[i][0]
        model[x][y]=1
        
    return model

In [ ]:
#Permet de trouver la main, le coude et l'épaule depuis un masks de points d'un bras
def attributsBras(b, h, w, tete, nage) :
    
    bras_trie = []
    nbPoints = len(b)
    
    dist_Min = 100000000000
    a_Dmin = 0
    dist_Max, a_Dmax, b_Dmax = 0, 0, 0
 
    #Recherche de l'épaule : le point le plus proche de la tête.
    for i in range(nbPoints):
        if ((b[i][0]-tete[1][0])**2+(b[i][1]-tete[1][1])**2 < dist_Min) :           
            dist_Min = (b[i][0]-tete[1][0])**2+(b[i][1]-tete[1][1])**2
            a_Dmin = i 
    print(a_Dmin)
    print(b)
    epaule=b[a_Dmin]

    
    bras_trie.append(epaule)
    #Si nage = dos, alors le bras est tendu, on ne cherche qu'un seul autre point : la main.
    dist_Max = -1
    a_Dmax = 0
    if nage == "DOS":
        for i in range(nbPoints):
            if ((b[i][0]-tete[1][0])**2+(b[i][1]-tete[1][1])**2 > dist_Max) :           
                dist_Max = (b[i][0]-tete[1][0])**2+(b[i][1]-tete[1][1])**2
                a_Dmax = i
        #le coude est à la moitié du bras.
        x_coude = b[a_Dmin][1]+int((b[a_Dmax][1]-b[a_Dmin][1])/2)
        y_coude = b[a_Dmin][0]+int((b[a_Dmax][0]-b[a_Dmin][0])/2)
        bras_trie.append([y_coude,x_coude])
        bras_trie.append(b[a_Dmax])
        
    #Si nage = nage libre ou papillon ou brasse
    #On cherche si le bras est devant ou derriere 
    bras_devant = False
    orientation = "gauche"
    if (tete[0][1] < w/2): # Si la nageur se déplace vers la gauche
        for i in range(1,len(b)):
            if (b[i][1] < tete[0][1]): #alors, si un point est à gauche de la tête, le bras est considéré comme devant
                bras_devant = True
    else: # Si la nageur se déplace vers la gauche
        orientation = "droite"
        for i in range(1,len(b)):
            if (b[i][1] > tete[0][1]): #si un point est à droite de la tête, le bras est considéré comme devant
                    bras_devant = True
    main = []
    if (bras_devant == False and orientation == "droite") or (bras_devant == True and orientation == "gauche"):
        x_min = 100000
        i_min = -1
        for i in range(nbPoints):
            if b[i][1]<x_min:
                x_min = b[i][1]
                i_min = i
        main = b[i_min]
    else:
        x_max = 0
        i_max = -1
        for i in range(nbPoints):
            if b[i][1]>x_max:
                x_max = b[i][1]
                i_max = i
        main = b[i_max]
    #  Le coude est le point le plus éloigné de l'épaule et de la main. On regarde le minimum entre les distances avec le coude
    # ou la main, le plus grand minimum de distance est le points du coude.
    dist_max = 0    
    i_coude = -1
    for i in range(nbPoints):
        dist_main = (b[i][0]-main[0])**2+(b[i][1]-main[1])**2
        dist_epaule = (b[i][0]-epaule[0])**2+(b[i][1]-epaule[1])**2
        dist = min(dist_main, dist_epaule)
        if dist > dist_max:
            dist_max = dist
            i_coude = i
    coude = b[i_coude]
    bras_trie.append(coude)
    bras_trie.append(main)
    print(orientation)
    print(bras_devant)
    return bras_trie

    

In [ ]:
#Permet de trouver le centre de la tête et la nuque depuis les points de la tête
def attributsTete (t, h, w):
    tete = []
    centre_Tete = []
    base_Tete = []
    
    xMoy = 0
    yMoy = 0
    
    for i in range (len(t)):
        xMoy = xMoy + t[i][0]
        yMoy = yMoy + t[i][1]
    xMoy = int(xMoy/len(t))
    yMoy = int(yMoy/len(t))
    
    centre_Tete.append(xMoy)
    centre_Tete.append(yMoy)
    
    val, valMax, iBase = 0, 0, 0
    gauche = False
    
    if t[i][1] < (w/2):
        gauche = True
        
    for i in range (len(t)):
        if gauche:
            val = t[i][1]
        else:
            val = w-t[i][1]
            
        if val > valMax:
            valMax = val
            iBase = i
            
    base_Tete.append(t[iBase][0])
    base_Tete.append(t[iBase][1])
    
    
    tete.append(centre_Tete)
    tete.append(base_Tete)
    
    return tete

In [ ]:
import skimage
image_path = ROOT_DIR + '/datasets/zoom_nageur/100 NL d Gwandju/nageur_1/'
savePathFolder = ROOT_DIR + '/datasets/position/100 NL d Gwandju/nageur_1/'
files = os.listdir(image_path)
nage = 'NL'

all_PointsCorps = []
all_PointsTete = []
all_PointsBras = []

for i in range(0,1500):
    if ('crop_'+str(i)+'.jpg') in files :
    
        img = skimage.io.imread(image_path+'crop_'+str(i)+'.jpg')
        img_arr = np.array(img)
        #Récupère les informations
        results = model.detect([img_arr], verbose=1)
        r = results[0]
        masks = r['masks']
        scores = r['scores']        
        classIds = r['class_ids']
        N = masks.shape[-1]
        if N>0:
            #Fonction pour visualiser la sortie du masks rcnn
            visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                    dataset_val.class_names, r['scores'], figsize=(5,5))
            
            #Fonctions permettant de visualiser les Features maps
            activations = model.run_graph([img], [
            ("input_image",        tf.identity(model.keras_model.get_layer("input_image").output)),
            ("res2c_out",          model.keras_model.get_layer("res2c_out").output),
            ("res3c_out",          model.keras_model.get_layer("res3c_out").output),
            ("rpn_bbox",           model.keras_model.get_layer("rpn_bbox").output),
            ("roi",                model.keras_model.get_layer("ROI").output),
            ])
            display_images(np.transpose(activations["res2c_out"][0,:,:,:4], [2, 0, 1]), cols=4)      
######################## HEAD ############################
            pointsTete = []        
            #On récupère le mask de tête qui a le plus gros score
            headScore = 0
            headId = -1
            for j in range(N):
                if (dataset_val.class_names[classIds[j]]=='head'):
                    if scores[j]>headScore:
                        headScore = scores[j]
                        headId = j
            if headId > -1:
                mask = masks[:,:,headId]
                #génère les points
                pointsTete = getPoints(mask)
                #tri les points
                #pointsTete = triTab(pointsTete, mask.shape[0], mask.shape[1])
                #Récupère le centre et la nuque
                pointsTete = attributsTete(pointsTete, mask.shape[0], mask.shape[1])
                all_PointsTete.append([i,pointsTete])
                centreTete = pointsTete[0]        
                x1 = centreTete[1]
                y1 = centreTete[0]

                nuque = pointsTete[1]
                x2 = nuque[1]
                y2 = nuque[0]

                #cv2.circle(img,(x2,y2), 3, (0,0,255), -1)
                cv2.circle(img,(x1,y1), 3, (0,255,0), -1)
                cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
    ###################### ARMS ############################
            if len(all_PointsTete) > 0:
                arms = []
                arms1Score = 0
                arms1Id = -1
                #arms avec les scores les plus élevé                            
                for j in range(N):
                    if (dataset_val.class_names[classIds[j]]=='arms'):
                        if scores[j]>arms1Score:
                            arms1Score = scores[j]
                            arms1Id = j


                arms2Score = 0
                arms2Id = -1
                for j in range(N):
                    if (dataset_val.class_names[classIds[j]]=='arms'):
                        if j != arms1Id:
                            if scores[j]>arms2Score:
                                arms2Score = scores[j]
                                arms2Id = j


                if arms1Id > -1:
                    mask = masks[:,:,arms1Id]
                    pointsBras = getPoints(mask)
                    #pointsBras = triTab(pointsBras, mask.shape[0], mask.shape[1])
                    tete = all_PointsTete[-1]
                    pointsBras = attributsBras(pointsBras, mask.shape[0], mask.shape[1],tete[1], nage)
                    x1 = pointsBras[0][1]
                    y1 = pointsBras[0][0]
                    cv2.circle(img,(x1,y1), 3, (255,255,255), -1)
                    x2 = pointsBras[1][1]
                    y2 = pointsBras[1][0]
                    cv2.circle(img,(x2,y2), 3, (255,255,0), -1)
                    x3 = pointsBras[2][1]
                    y3 = pointsBras[2][0]
                    cv2.circle(img,(x3,y3), 3, (255,0,0), -1)
                    xNuque = tete[1][1][1]
                    yNuque = tete[1][1][0]
                    cv2.line(img,(x1,y1),(x2,y2),(255,255,0),2)
                    cv2.line(img,(x2,y2),(x3,y3),(255,255,0),2)
                    if tete[0]==i:
                        cv2.line(img,(x1,y1),(xNuque,yNuque),(255,255,255),2)


                if arms2Id > -1:
                    mask = masks[:,:,arms2Id]
                    pointsBras = getPoints(mask)
                    #pointsBras = triTab(pointsBras, mask.shape[0], mask.shape[1])
                    tete = all_PointsTete[-1]
                    pointsBras = attributsBras(pointsBras, mask.shape[0], mask.shape[1],tete[1], nage)
                    x1 = pointsBras[0][1]
                    y1 = pointsBras[0][0]
                    cv2.circle(img,(x1,y1), 3, (255,255,255), -1)
                    x2 = pointsBras[1][1]
                    y2 = pointsBras[1][0]
                    cv2.circle(img,(x2,y2), 3, (255,255,0), -1)
                    x3 = pointsBras[2][1]
                    y3 = pointsBras[2][0]
                    cv2.circle(img,(x3,y3), 3, (255,0,0), -1)
                    xNuque = tete[1][1][1]
                    yNuque = tete[1][1][0]
                    cv2.line(img,(x1,y1),(x2,y2),(255,255,0),2)
                    cv2.line(img,(x2,y2),(x3,y3),(255,255,0),2)
                    if tete[0]==i:
                        cv2.line(img,(x1,y1),(xNuque,yNuque),(255,255,255),2)

                if N > 0:    
                    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                            dataset_val.class_names, r['scores'], figsize=(5,5))  
                    visualize.display_top_masks(img, r['masks'], r['class_ids'], dataset_val.class_names)

                    pathSave = savePathFolder+'/crop_'+str(i)+'.jpg'
                    cv2.imwrite(pathSave, cv2.cvtColor(img, cv2.COLOR_RGB2BGR)) 


In [ ]:
video_path = ROOT_DIR + '/datasets/position/100 NL d Gwandju/nageur_1/'
width = 260
height = 80
size = (width,height)
out = cv2.VideoWriter(ROOT_DIR + '/datasets/position/100 NL d Gwandju/nageur_1/zoom.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
files = os.listdir(video_path)

for i in range(0,1500):
    if ('crop_'+str(i)+'.jpg') in files :
        img=cv2.resize(cv2.imread(video_path+'crop_'+str(i)+'.jpg'),(width, height))
        out.write(img)
out.release()